In [ ]:
%matplotlib inline

In [ ]:
# my imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import datasets, classifiers and performance metrics
from sklearn import datasets, metrics, svm
from sklearn.model_selection import train_test_split

## Recognizing Handwritten digits